This command installs and updates several key Python libraries used for natural language processing and model optimization. The exclamation mark ! allows running shell commands directly from environments like Jupyter Notebook. The pip install command fetches and installs packages: transformers provides pre-trained models and training utilities from Hugging Face, datasets handles efficient dataset loading and preprocessing, accelerate optimizes multi-GPU and distributed training, ray[tune] enables scalable hyperparameter tuning, and optuna offers an alternative, efficient framework for automated hyperparameter optimization. The -U flag ensures all packages are upgraded to their latest compatible versions.

In [1]:
!pip install transformers datasets accelerate ray[tune] optuna -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.6/511.6 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.9/400.9 kB 23.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.2/102.2 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 20.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 MB 9.9 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 18.1.0
    Uninstalling pyarrow-18.1.0:
      Successfully uninstalled pyarrow-18.1.0
  Attempting uninstall: click
    Found existing installation: click 8.3.0
    Uninstalling click-8.3.0:
      Successfully uninstalled click-8.3.0
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take

**Function Description**

This script performs random hyperparameter search and fine-tuning for a binary RoBERTa-based text classifier on a mental-health Twitter dataset, with explicit adjustments to operate within a 5 GB GPU memory constraint. It prepares and tokenizes the dataset, defines a model initialization routine and evaluation metrics, samples hyperparameters using Optuna’s suggestion methods, executes a defined number of random trials via the Hugging Face Trainer, and then retrains and evaluates a final model using the best hyperparameters discovered by the random search.

**Syntax Explanation**

The code uses Hugging Face transformers and datasets with AutoTokenizer and AutoModelForSequenceClassification for model-agnostic loading, Optuna for randomized hyperparameter suggestions, and scikit-learn for metric computation. The dataset is converted from pandas to a Hugging Face Dataset and tokenized with truncation, padding, and a maximum sequence length of 128 tokens. The Trainer is instantiated with model_init so each trial starts from fresh pretrained weights, and trainer.hyperparameter_search is invoked with hp_space set to the random_hp_space function, which uses suggest_loguniform, suggest_categorical, suggest_float, and suggest_int to sample learning rate, batch size, gradient accumulation steps, weight decay, and number of epochs.

**Inputs**

The primary input is the CSV file Mental-Health-Twitter.csv containing post_text and label columns, where label is renamed to labels for Trainer compatibility. The script removes missing or empty post_text entries, performs a stratified train validation split, and samples the training set to 5,000 rows and the evaluation set to 1,000 rows to reduce memory usage. The pretrained model identifier margotwagner/roberta-psychotherapy-eval is used to initialize the tokenizer and model weights.

**Outputs**

The program prints device selection, dataset sizes, and label distributions, then reports the number of random trials to run and per-trial evaluation results logged through Trainer and Optuna. After the search completes, it prints the best trial and its hyperparameters, retrains a final model with those hyperparameters, evaluates that model, and saves the final checkpoint to ./final_model_mental_health_5gb_random. The evaluation metrics printed include loss, accuracy, F1-score, precision, and recall.

**Code Flow**

The script begins by importing libraries, setting a reproducible seed, and selecting the compute device. It loads and cleans the CSV dataset, renames the label column to labels, performs a stratified train validation split, and reduces the dataset size to 5,000 training and 1,000 evaluation samples. The DataFrames are converted to Hugging Face Dataset objects and tokenized with truncation, padding, and max_length=128, then formatted as PyTorch tensors. A model_init function is defined to instantiate a fresh AutoModelForSequenceClassification for each trial, and compute_metrics is defined to compute accuracy, F1, precision, and recall from predictions. The random_hp_space function defines the random search space tailored for a 5 GB GPU, sampling learning rate from a log-uniform range, selecting per-device batch sizes from conservative options, sampling gradient accumulation steps to permit effective larger batch sizes without increased VRAM, sampling weight decay from a float range, and sampling the number of epochs as an integer. TrainingArguments set shared training behaviors and the Trainer is created with model_init, datasets, tokenizer, and metrics. trainer.hyperparameter_search runs NUM_RANDOM_TRIALS random trials using Optuna and maximizes eval_f1 via the provided objective function. If a best trial is found, the script reconstructs TrainingArguments with the chosen hyperparameters, reinitializes a Trainer, trains the final model, evaluates it, and saves the checkpoint.

**Comments and Observations**

The script is well suited to constrained GPU environments because it reduces dataset size, limits per-device batch sizes, and introduces gradient accumulation steps to simulate larger effective batch sizes without exceeding memory limits. Using suggest_loguniform for learning rate is appropriate because it allows sampling across orders of magnitude, and model_init ensures trial independence so that each hyperparameter configuration is evaluated from the same starting point. Optimizing for F1 makes sense for binary labels that may be imbalanced, and keeping max_length at 128 is a reasonable choice for tweet-length inputs but should be validated against token length distribution. It is recommended to verify the existence of the __index_level_0__ column before removing it to avoid errors, to consider enabling Optuna pruning to stop low-performing trials early, and to validate that labels contains exactly 0 and 1 so that metric calculations remain meaningful.

In [2]:
# 1. SETUP AND INSTALLATION
# Run this command first in your Colab notebook:
# !pip install transformers datasets accelerate ray[tune] optuna pandas -U

import torch
import os
import numpy as np
import pandas as pd
from datasets import Dataset
from transformers import (
    AutoModelForSequenceClassification,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    set_seed
)
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import optuna # Import optuna to use its suggestion methods for random search

# Set a consistent seed for reproducibility across runs
set_seed(42)

# Ensure GPU is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU. For faster training, consider enabling a GPU runtime.")

# --- 2. DATA PREPARATION (Using your Mental-Health-Twitter.csv) ---

# Load your dataset
try:
    df = pd.read_csv("Mental-Health-Twitter.csv")
    print("Dataset loaded successfully.")
except FileNotFoundError:
    print("Error: 'Mental-Health-Twitter.csv' not found. Please upload it to your Colab environment.")
    exit()

# Filter out rows where 'post_text' is NaN or empty
df = df.dropna(subset=['post_text'])
df = df[df['post_text'].str.strip() != '']

# Rename 'label' to 'labels' for Hugging Face Trainer compatibility
df = df.rename(columns={"label": "labels"})

# Split data into training and validation sets
# IMPORTANT: Keep dataset size manageable for 5GB GPU.
# 10k train / 2k eval is still quite a lot for a 5GB GPU and RoBERTa-base.
# Let's reduce it further to make sure even batch_size=16 is stable.
train_df, eval_df = train_test_split(df, test_size=0.1, stratify=df['labels'], random_state=42)
train_df = train_df.sample(n=5000, random_state=42) # REDUCED TO 5K TRAINING SAMPLES
eval_df = eval_df.sample(n=1000, random_state=42)   # REDUCED TO 1K EVALUATION SAMPLES

print(f"Using {len(train_df)} training samples and {len(eval_df)} evaluation samples.")
print(f"Train label distribution:\n{train_df['labels'].value_counts(normalize=True)}")
print(f"Eval label distribution:\n{eval_df['labels'].value_counts(normalize=True)}")

# Convert pandas DataFrames to Hugging Face Dataset objects
train_dataset = Dataset.from_pandas(train_df[['post_text', 'labels']])
eval_dataset = Dataset.from_pandas(eval_df[['post_text', 'labels']])

# Initialize Tokenizer for your specific model
MODEL_NAME = "margotwagner/roberta-psychotherapy-eval"
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

def tokenize_function(examples):
    # Max length is already 128, which is good. Don't go higher.
    return tokenizer(examples["post_text"], truncation=True, padding=True, max_length=128)

# Apply tokenization
tokenized_train = train_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])
tokenized_eval = eval_dataset.map(tokenize_function, batched=True, remove_columns=["post_text", "__index_level_0__"])

# Set format to PyTorch tensors
tokenized_train.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])
tokenized_eval.set_format("torch", columns=['input_ids', 'attention_mask', 'labels'])


# --- 3. MODEL, METRICS, AND HYPERPARAMETER DEFINITION ---

# Function to initialize a fresh model for each grid search run
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(MODEL_NAME, num_labels=2).to(device)

def compute_metrics(p):
    preds = np.argmax(p.predictions, axis=1)
    acc = accuracy_score(p.label_ids, preds)
    f1 = f1_score(p.label_ids, preds, average="binary")
    precision = precision_score(p.label_ids, preds, average="binary")
    recall = recall_score(p.label_ids, preds, average="binary")
    return {"accuracy": acc, "f1": f1, "precision": precision, "recall": recall}

# --- HYPERPARAMETER SEARCH SPACE FOR RANDOM SEARCH (5GB GPU CONSCIOUS) ---
def random_hp_space(trial):
    """
    This function defines the hyperparameter search space for Random Search,
    optimized for a 5GB GPU memory limit.
    """
    # 1. Learning Rate (log-uniform distribution is common for learning rates)
    learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5) # Broader range

    # 2. Batch Size (CRITICAL FOR 5GB GPU)
    # We must be very conservative here. Batch size 32 is likely too large.
    # We will mainly stick to 8 or 16, and consider gradient_accumulation_steps.
    per_device_train_batch_size = trial.suggest_categorical("per_device_train_batch_size", [8, 16])

    # 3. Gradient Accumulation Steps (compensates for small batch sizes)
    # This effectively makes the 'effective' batch size larger without increasing VRAM.
    # effective_batch_size = per_device_train_batch_size * gradient_accumulation_steps
    gradient_accumulation_steps = trial.suggest_categorical("gradient_accumulation_steps", [1, 2, 4]) # Try accumulating gradients

    # 4. Weight Decay (uniform float distribution)
    weight_decay = trial.suggest_float("weight_decay", 0.0, 0.1, step=0.01)

    # 5. Number of Training Epochs (keep low for faster trials)
    num_train_epochs = trial.suggest_int("num_train_epochs", 2, 3) # Max 3 epochs

    return {
        "learning_rate": learning_rate,
        "per_device_train_batch_size": per_device_train_batch_size,
        "gradient_accumulation_steps": gradient_accumulation_steps, # New HP
        "weight_decay": weight_decay,
        "num_train_epochs": num_train_epochs,
    }


# --- 4. TRAINING ARGUMENTS (Fixed for all runs) ---
training_args = TrainingArguments(
    output_dir="./random_search_results_mental_health_5gb", # New output directory
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",
    fp16=torch.cuda.is_available(), # Enable mixed precision for T4 GPU
    report_to="none",
    num_train_epochs=3, # Placeholder, will be suggested by random_hp_space
    warmup_steps=100,
    logging_dir="./logs_random_5gb", # New logging directory
    logging_steps=500,
    dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    # gradient_accumulation_steps will be passed directly from random_hp_space
)

# Initialize the Trainer
trainer = Trainer(
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_eval,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

# --- Define the objective function for Optuna ---
def optuna_hp_objective(metrics):
    """
    Optuna objective function that returns the F1 score for maximization.
    `metrics` is the dictionary returned by trainer.evaluate().
    """
    return metrics["eval_f1"]


# --- 5. EXECUTION OF RANDOM SEARCH ---
print("\n--- Starting Random Search (using Optuna backend) ---")
print(f"Optimizing for '{training_args.metric_for_best_model}' score...")

NUM_RANDOM_TRIALS = 30 # Aim for 30 trials to explore more thoroughly given the reduced epoch count

print(f"Number of random trials to run: {NUM_RANDOM_TRIALS}")
print("NOTE: Batch sizes are kept low and gradient accumulation is used to manage 5GB GPU memory.")
print("      Dataset size has also been reduced for quicker iteration.")

best_trial = trainer.hyperparameter_search(
    backend="optuna",
    hp_space=random_hp_space, # Use the new random_hp_space function
    direction="maximize",
    n_trials=NUM_RANDOM_TRIALS,
    compute_objective=optuna_hp_objective,
)

print("\n--- Random Search Complete ---")
print("\nBEST HYPERPARAMETERS FOUND:")

# Extract and print the best configuration
if best_trial:
    print(best_trial)
    best_hps = best_trial.hyperparameters
    print("\nBest Hyperparameters:")
    for key, value in best_hps.items():
        print(f"  {key}: {value}")
    print(f"\nBest Metrics (on evaluation set): {best_trial.metrics}")
else:
    print("Search failed or no best trial found.")

print("\n--- Final Step: Train a model with the best hyperparameters ---")
if best_trial:
    final_training_args = TrainingArguments(
        output_dir="./final_model_mental_health_5gb_random", # New output directory
        eval_strategy="epoch",
        save_strategy="epoch",
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        fp16=torch.cuda.is_available(),
        report_to="none",
        num_train_epochs=best_hps["num_train_epochs"],
        per_device_train_batch_size=best_hps["per_device_train_batch_size"],
        gradient_accumulation_steps=best_hps["gradient_accumulation_steps"], # Apply best G.A.S.
        learning_rate=best_hps["learning_rate"],
        weight_decay=best_hps["weight_decay"],
        warmup_steps=100,
        logging_dir="./final_logs_random_5gb", # New logging directory
        logging_steps=500,
        dataloader_num_workers=os.cpu_count() // 2 if os.cpu_count() else 0,
    )

    final_trainer = Trainer(
        model_init=model_init,
        args=final_training_args,
        train_dataset=tokenized_train,
        eval_dataset=tokenized_eval,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )

    print("\nTraining final model with best hyperparameters from Random Search (5GB GPU config)...")
    final_trainer.train()

    print("\nFinal model training complete. Best model saved to './final_model_mental_health_5gb_random'.")
    metrics = final_trainer.evaluate()
    print(f"Evaluation metrics of the final model: {metrics}")
else:
    print("No best hyperparameters found, skipping final model training.")

Using GPU: Tesla T4
Dataset loaded successfully.
Using 5000 training samples and 1000 evaluation samples.
Train label distribution:
labels
1    0.5
0    0.5
Name: proportion, dtype: float64
Eval label distribution:
labels
0    0.507
1    0.493
Name: proportion, dtype: float64


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/394 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

...

[I 2025-11-08 06:42:40,616] Trial 28 pruned. 
/tmp/ipython-input-3785949678.py:107: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  learning_rate = trial.suggest_loguniform("learning_rate", 1e-5, 5e-5) # Broader range


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,No log,0.424501,0.805000,0.805583,0.792157,0.819473


[I 2025-11-08 06:43:23,432] Trial 29 pruned. 



--- Random Search Complete ---

BEST HYPERPARAMETERS FOUND:
BestRun(run_id='13', objective=0.8897795591182365, hyperparameters={'learning_rate': 3.688694822915927e-05, 'per_device_train_batch_size': 16, 'gradient_accumulation_steps': 1, 'weight_decay': 0.07, 'num_train_epochs': 3}, run_summary=None)

Best Hyperparameters:
  learning_rate: 3.688694822915927e-05
  per_device_train_batch_size: 16
  gradient_accumulation_steps: 1
  weight_decay: 0.07
  num_train_epochs: 3


AttributeError: 'BestRun' object has no attribute 'metrics'